In [1]:
import numpy as np
import os
import time

from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn import mixture, metrics

from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk
from skimage import color
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob
from radio_beam.commonbeam import getMinVolEllipse

from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset


In [ ]:
def img_load_preprocess(path, img_names, h=150, k=1):
    imgs = []
    for i, class_img_names in enumerate(img_names):
        imgs.append([])
        for name in class_img_names:
            img = io.imread(path + '/' + name)
            img = grainPreprocess.image_preprocess(img, h, k)
            imgs[i].append(img)
    return imgs

In [ ]:
path = 'data/images'
all_images = np.load('data/saved np/all_images.npy', allow_pickle=True)

grain_names = np.array(
    [['Ultra_Co8.jpg'], ['Ultra_Co11.jpg'], ['Ultra_Co6_2.jpg'], ['Ultra_Co15.jpg'], ['Ultra_Co25.jpg']])
types = ['средние зерна', 'мелкие зерна', 'мелкие зерна', 'крупные зерна', 'средне-мелкие зерна']
grain_images = img_load_preprocess(path, grain_names)

### Считаем все изображения, предобработаем их и сохраним

In [5]:
# images_dir='../data/dataset_4x/'
# images_dir='images/artif/'
# save_name='images_4x_(512,512,1).npy'
# save_name='artif.npy'

images_dir = '../data/dataset_sub_base_left/'
save_name = '(1500,1500,1)_sub_base_left'

all_images = grainPreprocess.read_preprocess_data(images_dir=images_dir, save_name=save_name,
                                                  max_images_num_per_class=90,
                                                  preprocess=True, save=True, crop_bottom=False, resize=True,
                                                  resize_shape=(1500, 1500, 1))

../data/dataset_sub_base_left\Ultra_Co11\Ultra_Co11-1.jpeg---| 0.0% Complete ETA:None s

../data/dataset_sub_base_left\Ultra_Co11\Ultra_Co11-100.jpeg-| 0.2% Complete ETA:4028.1 s



KeyboardInterrupt: 

In [7]:
image = io.imread('../data/dataset_sub_base_left/Ultra_Co8/Ultra_Co8-001.jpeg').astype(np.uint8)

### Сгенерируем распределения углов, аппроксимируем их бимодальным гауссом и сохраним

In [ ]:
from src.utils import grainGenerate, GrainLogs, grainMark, grainStats, grainApprox
import numpy as np

In [ ]:

# images = np.load('../data/saved np/images_4x_preprocess.npy')
# images = np.load('artif.npy')

images = np.load('images_left_(1500,1500,1)_preprocessed.npy')
names = np.load('metadata_left_(1500,1500,1)_preprocessed.npy')
names = [name.split('\\')[-1] for name in names]

folder = 'data/'

types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}


# path = 'data/saved np'
path = 'images/saved np/'
# name = 'artif'
name = 'left_(1500,1500,1)'


# grain_names =['cube compress', 'magnet']
# types = ['-', '-', ]

for step in range(1, 6):
    grainGenerate.angles_approx_save(path, images, name, names, types_dict, step, save=True)


In [ ]:
def angles_approx(folder, name, step, width, height, plot_name=None, y_lim=None, save=False,indeces=None, font_size=20):
    #
    # вывод распределения углов для всех фотографий одного образца
    #

    xy_scatter = np.load(f'{folder}/' + CfgAnglesNames.values + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_gauss = np.load(f'{folder}/' + CfgAnglesNames.approx + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_gauss_data = np.load(f'{folder}/' + CfgAnglesNames.approx_data + f'{name}' + f'{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/' + CfgAnglesNames.legend + f'{name}' + f'{step}.npy', allow_pickle=True)

    if indeces is not None:
        xy_scatter = xy_scatter[indeces]
        xy_gauss = xy_gauss[indeces]
        xy_gauss_data = xy_gauss_data[indeces]
        texts = texts[indeces]

    plt.rcParams['font.size'] = '15'
    gs = gridspec.GridSpec(1, 1)
    plt.figure(figsize=(width, height))

    ax = plt.subplot(gs[0])
    # маркеры для одновременного вывода скаттера для разных классов
    # количество варкеров=количество классов-1
    markers = ['v', 's', 'D', 'd', 'p', '*']
    # colors = ['aqua', 'orange', 'green', 'red', 'indigo']

    for i in range(len(xy_scatter)):
        ax.plot(xy_gauss[i][0], xy_gauss[i][1],)

    ax.legend(texts, fontsize=14, loc=1)

    for i in range(len(xy_scatter)):
        marker = markers[i]
        ax.scatter(xy_scatter[i][0], xy_scatter[i][1], marker=marker)

    plt.ylabel('p(x)', fontsize=font_size)
    plt.xlabel('угол связующего вещества, градусы', fontsize=font_size)

    if y_lim:
        plt.ylim(0, y_lim)
    # plt.autoscale(enable=True, axis='y')

    if save:
        plt.savefig(f'распределение_углов_{plot_name}_шаг_{step}')

    plt.show()

In [ ]:
# folder = 'data/'
# path = 'images/saved np/'
path = '../data/saved np/'

step = 5
width = 20
height = 20
# name = '4x_data_border_fix_'
# name = 'artif'
name = ''
# y_lim = 0.015
# y_lim = 0.013
y_lim = None
# save = True
save = True
plot_name = ''

# генерация распределения, аппроксимирование распределения и сохранение данных
angles_approx(path, name, step, width, height, plot_name=plot_name, y_lim=y_lim, save=save, indeces=[0,2,3])

In [ ]:
def angles_approx(folder, step, width, height, save=False, font_size=20):
    #
    # вывод распределения углов для всех фотографий одного образца
    #

    xy_scatter = np.load(f'{folder}/' + CfgAnglesNames.values + f'{step}.npy', allow_pickle=True)
    xy_gauss = np.load(f'{folder}/' + CfgAnglesNames.approx + f'{step}.npy', allow_pickle=True)
    xy_gauss_data = np.load(f'{folder}/' + CfgAnglesNames.approx_data + f'{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/' + CfgAnglesNames.legend + f'{step}.npy', allow_pickle=True)

    synth_angles = np.load(f'{folder}/wc-co.npy')
    synth_angles = np.concatenate([synth_angles, np.zeros((180))])

    plt.rcParams['font.size'] = '15'
    gs = gridspec.GridSpec(1, 1)
    plt.figure(figsize=(width, height))

    ax = plt.subplot(gs[0])
    # маркеры для одновременного вывода скаттера для разных классов
    # количество варкеров=количество классов-1
    markers = ['v', 's', 'D', 'd', 'p', '*']
    for i in range(len(xy_scatter)):
        marker = markers[i]
        ax.plot(xy_gauss[i][0], xy_gauss[i][1])
        ax.scatter(xy_scatter[i][0], xy_scatter[i][1], marker=marker)

    ax.plot(synth_angles[::-1])

    texts = list(texts)
    texts.append('синтетические углы')
    ax.legend(texts, fontsize=14, loc=2)
    plt.ylabel('p(x)', fontsize=font_size)
    plt.xlabel('угол связующего вещества, градусы', fontsize=font_size)
    if save:
        plt.savefig(f'наложение {step}')

    plt.show()

In [ ]:
folder = 'data/saved np/'
step = 5
width = 20
height = 20

# генерация распределения, аппроксимирование распределения и сохранение данных
angles_approx(folder, step, width, height, save=True)

In [ ]:
def angles_approx_modes(folder, step, start1, stop1, start2, stop2, width, height, font_size=25):
    #
    # аппроксимация каждой моды
    #
    xy_scatter_o = np.load(f'{folder}/xy_scatter_step_{step}.npy', allow_pickle=True)
    xy_gauss_o = np.load(f'{folder}/xy_gauss_step_{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/texts_step_{step}.npy', allow_pickle=True)

    xy_scatter1 = []
    xy_scatter2 = []

    for j, sc in enumerate(xy_scatter_o):
        x = sc[0]
        y = sc[1]
        xy_scatter1.append([[], []])
        xy_scatter2.append([[], []])
        for i, xi in enumerate(x):
            if xi <= stop1 and xi >= start1:
                xy_scatter1[j][0].append(x[i])
                xy_scatter1[j][1].append(y[i])

            if xi <= stop2 and xi >= start2:
                xy_scatter2[j][0].append(x[i])
                xy_scatter2[j][1].append(y[i])

    plt.rcParams['font.size'] = '25'
    markers = ['v', 's', 'D', 'd', 'p', '*']
    k = 2
    for i in range(len(xy_scatter_o)):
        plt.figure(figsize=(width, height))
        marker = markers[i]

        #    plt.scatter(xy_scatter_o[i][0],xy_scatter_o[i][1])

        x1 = np.array(xy_scatter1[i][0])
        y1 = xy_scatter1[i][1]

        x2 = np.array(xy_scatter2[i][0])
        y2 = xy_scatter2[i][1]

        mu1, sigma1, amp1 = grainApprox.gaussian_fit(x1, y1, mu=120)
        mu2, sigma2, amp2 = grainApprox.gaussian_fit(x2, y2, mu=225)

        x_gauss1 = np.arange(mu1 - 3 * sigma1, mu1 + 3 * sigma1)
        x_gauss2 = np.arange(mu2 - 3 * sigma2, mu2 + 3 * sigma2)

        y_gauss1 = grainStats.gaussian(x_gauss1, mu1, sigma1, amp1)
        y_gauss2 = grainStats.gaussian(x_gauss2, mu2, sigma2, amp2)

        #    plt.plot(x_gauss,y_gauss)
        plt.scatter(x1, y1, marker=marker)
        plt.plot(x_gauss1, y_gauss1)

        plt.scatter(x2, y2, marker=markers[i + 1])
        plt.plot(x_gauss2, y_gauss2)

        #  print('mu1=',mu1,' sigma1=',sigma1,'amp1=',amp1)
        #  print('mu2=',mu2,' sigma2=',sigma2,'amp2=',amp2)
        plt.legend(['moda1', 'moda2', 'approx1', texts[i]], fontsize=14, loc=1)
        plt.ylabel('p(x)', fontsize=font_size)
        plt.xlabel('угол связующего вещества, градусы', fontsize=font_size)
        # plt.ylim([0,max(np.log(xy_scatter[i][1][:-1]))])
    # plt.savefig('крупные моды отдельная аппрокс  ' + names[i])

    #  print(texts)

    #  plt.savefig('распределение углов')
    plt.show()

In [ ]:
folder = 'saved np'
width = 12
height = 15

step = 1

start = 340
stop = 360

start1 = 50
stop1 = 180

start2 = 180
stop2 = 300

angles_approx_modes(folder, step, start1, stop1, start2, stop2, width, height)